# Generating synthetic data sets of a river bed column forced by monoperiodic river and aquifer temperature signals

2024 adapted by Nicolas Flipo from demo_gen_test.ipynb created by Guillaume de Rochefort during MOLONARI 2022 

L'objet de cette démo est de présenter une classe chargée de générer une simulation des données des capteurs de pression et de température dans le cas où l'on ne disposerait pas de données issues du projet CAPTEUR. Based on the ``Time_series`` class in ``gen_test.py``

Nous montrerons l'objet Time_Series qui permet de générer des jeux de données purements périodique avec possibilité de bruit gaussien. Nous illustrerons l'interaction de Time_series avec la classe Column et la classe layer. 

Deux autres fichiers de démo mettent plutôt en valeur une utilisation globales des classes de gen_test.py et val_analy.py.


In [129]:
# démo gen_test
from pyheatmy import *
import matplotlib.pyplot as plt
import numpy as np




## 1. Cas sans perturbation

### 1.1 Paramètrage des signaux
Les signaux sont périodiques purs et il sagit de choisir l'amplitude, la période et la valeur moyenne des signaux de température de la rivière, de l'aquifère et de pression (les conditions limites). Les capteurs T1, T2, T3 du shaft ne sont générés que grâce au modèle direct.

Une amélioration possible de cette classe est de générer des signaux multipériodiques à la manière des listes de paramètres de layer, on donnerait une listes des paramètres des différentes fréquences voulues (diurne, mensuel, annuel, etc...)

In [ ]:
# l'utilisateur saisit ses paramètres d'émulation de mesure
"""Fenêtre temporelle"""
# le format demandé est celui-ci : (y,m,d,h,mn,s) compatible avec la librairie datetime
t_debut = (2011, 8, 1)
t_fin = (2011, 8, 31, 23, 59, 59)
dt = 15*NSECINMIN # pas de temps en (s) # A AMELIORER AVEC UN CONVERTISSEUR AUTOMATIQUE D'UNITES

"""Conditions limites"""
# Température de la rivière
zeroT = 0
#zeroT += ZERO_CELSIUS  #gen_test works only with forcings in celsius. See if it is deeper in pyheatmy or not

T_riv_amp = 5
T_riv_offset = 20  + zeroT
nday = 3
P_T_riv = nday*NHOURINDAY*4*dt #monthly   period
# Température de l'aquifère
T_aq_amp = 0
T_aq_offset = 14 + zeroT
P_T_aq = -9999 # à mettre dans le init
# Pression différentielle
dH_amp = 0
dH_offset = 0.1 #1meter ?
P_dh = -9999 #14*24*4*dt


depth_sensors = [.1, .2, .3, .4]

"""Bruit de mesure"""
sigma_meas_P = 0.1
sigma_meas_T = 1.

print("dt={0:.1f}s".format(dt))

### 1.2 Instanciation de l'objet
L'objet généré aura des valeurs par défaut, il faut utiliser une série de méthode pour mettre à jour ses paramètres essentiels.

#### 1.2.1 Création de l'objet

In [ ]:
"""Instanciation de l'objet Time_series"""
# un dictionnaire qui facilite le paramétrage avec des variables globales définies plus haut
time_series_dict_user1 = {
    "offset":.0,
    "depth_sensors":depth_sensors,
	"param_time_dates": [t_debut, t_fin, dt], 
    "param_dH_signal": [dH_amp, P_dh, dH_offset], #En vrai y aura une 4e valeur ici mais ca prendra en charge pareil
	"param_T_riv_signal": [T_riv_amp, P_T_riv, T_riv_offset],
    "param_T_aq_signal": [T_aq_amp, P_T_aq, T_aq_offset],
    "sigma_meas_P": sigma_meas_P,
    "sigma_meas_T": sigma_meas_T, #float
}
# instanciation du simulateur de données
emu_observ_test_user1 = Time_series.from_dict(time_series_dict_user1)

print("Checking params time series")
print("param_time_dates:", emu_observ_test_user1._param_dates)
print("param_dH_signal:", emu_observ_test_user1._param_dH)
print("param_T_riv_signal:", emu_observ_test_user1._param_T_riv)
print("param_T_aq_signal:", emu_observ_test_user1._param_T_aq)
print("sigma_meas_P:", emu_observ_test_user1._sigma_P)
print("sigma_meas_T:", emu_observ_test_user1._sigma_T)


#### 1.2.2 Instanciation des series de données temporelles aux limites sur la base des dates de simulation 

In [ ]:
emu_observ_test_user1._generate_all_series()



#### 1.2.3 Génération des points T1, T2, T3 avec le modèle direct

Il faut d'abord créer un objet colonne qui représente l'expérience

In [133]:
# l'utilisateur génère un dictionnaire avec les données importantes de la colonne
name ="Couche en sable"
zLow = 0.4
moinslog10IntrinK = 15
n = 0.1
lambda_s = 2 # test cas purement advectif
rhos_cs = 4e6

# modèle une couche
layers_list= layersListCreator([(name, zLow, moinslog10IntrinK, n, lambda_s, rhos_cs)])

# on utilise les mesures générées précédemment dans les init "dH_measures" et "T_measures"
col_dict = {
	"river_bed": 1., 
    "depth_sensors": depth_sensors, #En vrai y aura une 4e valeur ici mais ca prendra en charge pareil
	"offset": .0,
    "dH_measures": emu_observ_test_user1._molonariP_data,
    "T_measures": emu_observ_test_user1._T_Shaft_measures,
    "sigma_meas_P": None, #float
    "sigma_meas_T": None, #float
}
col = Column.from_dict(col_dict)

On peut vérifier que l'objet colonne a récupéré les valeurs d'initialisation de Time_series. 
On a également des conditions initiales [T1, T2, T3](t=0) qui viennent d'une méthode d'interpolation en ligne brisée


On utilise la méthode [_measures_column_one_layer(col, layers_list, nb_cells)] qui met à jour les mesures de T1, T2, T3 à chaque instant. Les objets Column et Time_series sont mis à jour ensemble.

In [ ]:
# on résout le modèle direct avec les observations simulées
nb_cells = 100
emu_observ_test_user1._measures_column_one_layer(col, layers_list, nb_cells)

In [ ]:
#import os
col.print_in_file_processed_MOLONARI_dataset()

In [ ]:
print("methode column verifiee")
col.plot_temperature_at_sensors()


In [ ]:
nt = len(col._temperatures[0,:])
dplot=15
col.plot_temperatures_umbrella(round(nt/dplot))



In [ ]:
flows = col.get_flows_solve()
unitLeg="m/s"
title="Débits"
col.plot_it_Zt(flows,title,unitLeg,1.04,2)

In [ ]:
temperatures = col.get_temperatures_solve()
unitLeg="°C"
title="Températures"
col.plot_it_Zt(temperatures,title,unitLeg)

In [ ]:
col.plot_CALC_results()

## 2. Cas avec perturbation des observations

In [141]:
emu_observ_test_user1._generate_all_series()

In [ ]:
# les sorties des différents capteurs après passage dans le modèle direct
nt = 3000
plt.plot(emu_observ_test_user1._T_riv_perturb[:nt], label="Triv")
n_sens = len(emu_observ_test_user1._T_Shaft_perturb[0])
for i in range(n_sens-1):
    plt.plot(emu_observ_test_user1._T_Shaft_perturb[:nt,i], label="T{}".format(i+1))
plt.plot(emu_observ_test_user1._T_Shaft_perturb[:nt,n_sens-1], label="Taq")
plt.legend()
plt.grid()

# Test valeur par défaut de la classe émulation

In [143]:
time_series_dict_user3 = {
    "offset" : 0,
    "depth_sensors":depth_sensors,
}

In [ ]:
emu_observ_test_user3 = Time_series.from_dict(time_series_dict_user3)

emu_observ_test_user3._generate_Shaft_Temp_series()
emu_observ_test_user3._T_Shaft_measures

# Test du cas multilayer 

In [145]:
t_debut = (2011, 8, 1)
t_fin = (2011, 8, 31, 23, 59, 59)
dt = 15*60 # pas de temps en (s)

T_riv_amp = 2
T_riv_offset = 20
P_T_riv = 72*4*dt

T_aq_amp = 0
T_aq_offset = 14
P_T_aq = 9999 # à mettre dans le init

dH_amp = 0
dH_offset = 1
P_dh = -9999 #14*24*4*dt

depth_sensors = [.1, .15, .25, .3, .4] # on a mis 5 capteurs dans le shaft au lieu de 4 pour montrer que le code s'adapte

In [146]:
time_series_dict_user4 = {
    "offset" : 0,
    "depth_sensors":depth_sensors,
	"param_time_dates": [t_debut, t_fin, dt], 
    "param_dH_signal": [dH_amp, P_dh, dH_offset], #En vrai y aura une 4e valeur ici mais ca prendra en charge pareil
	"param_T_riv_signal": [T_riv_amp, P_T_riv, T_riv_offset],
    "param_T_aq_signal": [T_aq_amp, P_T_aq, T_aq_offset],
    "sigma_meas_P": 0.05,
    "sigma_meas_T": 0.1, #float
}

In [ ]:
emu_observ_test_user4 = Time_series.from_dict(time_series_dict_user4)
emu_observ_test_user4._generate_all_series()

In [148]:
# l'utilisateur génère un dictionnaire avec les données importantes de la colonne
name = ["gravier","Couche en sable"]
zLow = [.2,.4]
moinslog10IntrinK = [4,10]
n = [.4,.1]
lambda_s = [2,2] 
rhos_cs = [4e6,4e6]

# on utilise les mesures générées précédemment
col_dict4 = {
	"river_bed": 1., 
    "depth_sensors": depth_sensors, #En vrai y aura une 4e valeur ici mais ca prendra en charge pareil
	"offset": .0,
    "dH_measures": emu_observ_test_user4._molonariP_data,
    "T_measures": emu_observ_test_user4._T_Shaft_measures,
    "sigma_meas_P": emu_observ_test_user4._sigma_P, #float
    "sigma_meas_T": emu_observ_test_user4._sigma_T, #float
}

col4 = Column.from_dict(col_dict4)

# modèle une couche
layers_list4= layersListCreator([(name[i], zLow[i], moinslog10IntrinK[i], n[i], lambda_s[i], rhos_cs[i]) for i in range(len(name))])

In [ ]:
nb_cells = 100
emu_observ_test_user4._measures_column_one_layer(col4, layers_list4, nb_cells)

# emu_observ_test_user4._generate_perturb_T_riv_dH_series()
# emu_observ_test_user4._generate_perturb_Shaft_Temp_series()

In [ ]:
emu_observ_test_user4._T_Shaft_perturb

In [ ]:
# les sorties des différents capteurs après passage dans le modèle direct
nt = 3000
plt.plot(emu_observ_test_user4._T_riv_perturb[:nt], label="Triv")
n_sens = len(emu_observ_test_user4._T_Shaft_perturb[0])
for i in range(n_sens-1):
    plt.plot(emu_observ_test_user4._T_Shaft_perturb[:nt,i], label="T{}".format(i+1))
plt.plot(emu_observ_test_user4._T_Shaft_perturb[:nt,n_sens-1], label="Taq")
plt.legend()
plt.grid()

In [ ]:
nt = len(col4._temperatures[0,:])
for i in range(1000,2000):
    plt.plot(col4._temperatures[:,i], -col._z_solve)
plt.ylabel("depth")
plt.xlabel("T (°C)")
plt.grid()
#plt.xlim(282,286)

In [ ]:
plt.imshow(col._temperatures[:,:], aspect='auto')